# How to Convert MCD19A2 HDF-EOS Data to Cloud Optimized GeoTIFFs (COGs)

**Summary**  

With the transition to NASA's Earthdata Cloud, all MODIS collections previously hosted in LP DAAC's on-premise Data Pool are now available in a modern, cloud-accessible environment. However, many of these collections—such as MCD19A2 are still distributed in the legacy HDF-EOS format.
While HDF-EOS remains a widely used format within the Earth science community, it was not designed for cloud-native access. As a result, working with HDF-EOS files using cloud-optimized tools (e.g., on-demand streaming, partial reads, or parallel processing) is inefficient and often cumbersome. Moreover, HDF-EOS can be difficult for new users to navigate, especially when extracting specific variables or subsetting data.
For a user-friendly access method, we recommend tools like [AppEEARS](https://appeears.earthdatacloud.nasa.gov/) (**if available**), which simplifies access to MODIS data through subsetting, reformatting, and spatial/temporal filtering. The **MCD19A2** MODIS product is not currently available through AppEEARS, which limits options for web-based subsetting and reformatting.This notebook demonstrates a programmatic approach to search, download, read and convert the MCD19A2 HDF-EOS data layers to cloud-optimized geotiffs, a more modern user-friendly data format.

**Background** 

The [MCD19A2](https://doi.org/10.5067/MODIS/MCD19A2.061) data product is a Moderate Resolution Imaging Spectroradiometer (MODIS) Terra and Aqua combined **Land Aerosol Optical Depth (AOD)** gridded Level 2 product produced daily at 1 kilometer (km) pixel resolution. The MCD19A2 product provides the atmospheric properties and view geometry over land and water at 5 km resolution. 

The MCD19A2 AOD data product contains the following Science Dataset (SDS) layers: blue band AOD at 0.47 µm, green band AOD at 0.55 µm, AOD uncertainty, fine mode fraction over water, column water vapor over land and clouds, smoke injection height, AOD QA, Angstrom Exponent over the ocean, cosine of solar zenith angle, cosine of view zenith angle, relative azimuth angle, scattering angle, and glint angle. Each SDS layer within each MCD19A2 Hierarchical Data Format 4 (HDF4) file contains a third dimension that represents the number of orbit overpasses. This factor could affect the total number of bands for each SDS layer. 

**Requirements**
 - [NASA Earthdata Account](https://urs.earthdata.nasa.gov/home)   
 - See the [set up instructions](https://github.com/nasa/LPDAAC-Data-Resources/blob/main/setup/setup_instructions_python.md) to set up a local compatible Python environment 


Import the required Python packages. 

In [1]:
import earthaccess
import rioxarray as rxr
import hvplot.xarray
from osgeo import gdal

### Search for and Download Data

To search for data, we'll use the [`earthaccess`](https://www.earthdata.nasa.gov/data/tools/earthaccess) Python library. `earthaccess` simplifies the amount of code required to search for data, and handles authentication. You do not need to authenticate to search, but do for downloading or streaming data. We'll go ahead and use the `login` function to add our credentials to this session. This function will retrieve your login info from a `.netrc` file if one exists, or prompt you for username and password and create one if you use the `persist` argument.

In [2]:
earthaccess.login(persist=True)

Next, the search parameters are defined and `earthaccess.search_data` is used to retrieve MODIS MCD19A2 (Version 061) data available in the NASA LP DAAC Cloud (`LPCLOUD`) archive for the provided parameters. 
To learn more about how to use `earthaccess` library see: <https://github.com/nasa/LPDAAC-Data-Resources/blob/main/python/tutorials/earthaccess_introduction.ipynb>


In [3]:
# Search
search_params = {
    "short_name":"MCD19A2",
    "version": "061",
    "provider": "LPCLOUD",
    "temporal": ('2025-03-01','2025-03-01'),
    "bounding_box": (-113.7505, 24.76229, -76.67831, 38.33684),  
    "count": 1000
}

results = earthaccess.search_data(**search_params)
len(results)

9

The source MDC19A2 files have HDF-EOS data format. Although the data is stored in the Earthdata Cloud The returned granules can be downloaded all at once or individually, which is useful when local disk space is limited. here, we only download the first granule from our response to `/data` folder.


In [4]:
# # To download all the files 
# downloaded_files = earthaccess.download(results, local_path='../data')

# # To only download one file at the time
downloaded_files = earthaccess.download(results[0], local_path='../data')

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
file = downloaded_files[0]
file

'..\\data\\MCD19A2.A2025060.h09v06.061.2025063025255.hdf'

### Open the File and Extract Metadata

Next, we open the downloaded file using the `rioxarray` library. The MCD19A2 product is organized into two main groups of variables based on spatial resolution:

- Variables with 1 km spatial resolution

- Variables with 5 km spatial resolution

These groups are stored as separate subdatasets within the HDF-EOS file, and each must be accessed separately.

The MCD19 SDS layers are three dimensional that is why data has a band dimention more than 1 (`band=4`). The total number of bands in each file is contingent upon the number of orbit overpasses (1-2 bands at the equator and up to 30 bands in polar regions). Note that at high latitudes, only the first 16 orbits with the largest coverage are selected for processing per day in order to limit the file size. Orbit information is stored in the `Orbit_amount` and `Orbit_time_stamp` attributes in the HDF file. 

In [6]:
ds = rxr.open_rasterio(file)

In [7]:
ds[0]

<xarray.Dataset> Size: 115MB
Dimensions:              (band: 4, x: 1200, y: 1200)
Coordinates:
  * band                 (band) int64 32B 1 2 3 4
  * x                    (x) float64 10kB -1.001e+07 -1.001e+07 ... -8.896e+06
  * y                    (y) float64 10kB 3.335e+06 3.334e+06 ... 2.224e+06
    spatial_ref          int64 8B 0
Data variables:
    Optical_Depth_047    (band, y, x) int16 12MB ...
    Optical_Depth_055    (band, y, x) int16 12MB ...
    AOD_Uncertainty      (band, y, x) int16 12MB ...
    Column_WV            (band, y, x) int16 12MB ...
    AngstromExp_470-780  (band, y, x) int16 12MB ...
    AOD_QA               (band, y, x) uint16 12MB ...
    FineModeFraction     (band, y, x) float32 23MB ...
    Injection_Height     (band, y, x) float32 23MB ...
Attributes: (12/85)
    ADDITIONALLAYERS:                   3
    ALGORITHMPACKAGEACCEPTANCEDATE:     TBD
    ALGORITHMPACKAGEMATURITYCODE:       Preliminary
    ALGORITHMPACKAGENAME:               MOD_PR19
    ALGORITHMPACKAGEVERSION:            2.0
    ASSOCIATEDINSTRUMENTSHORTNAME.1:    MODIS
    ...                                 ...
    TileID:                             51009006
    TOTALADDITIONALOBSERVATIONS:        851908
    TOTALOBSERVATIONS:                  6837412
    VERSIONID:                          61
    VERTICALTILENUMBER:                 6
    WESTBOUNDINGCOORDINATE:             -103.923048441979

Get the number of orbits/bands.

In [8]:
orbit_num = ds[0]['band'].values.tolist()
orbit_num

[1, 2, 3, 4]

You can also view data from only the first orbit.

In [9]:
ds[0].sel(band=1)

<xarray.Dataset> Size: 29MB
Dimensions:              (x: 1200, y: 1200)
Coordinates:
    band                 int64 8B 1
  * x                    (x) float64 10kB -1.001e+07 -1.001e+07 ... -8.896e+06
  * y                    (y) float64 10kB 3.335e+06 3.334e+06 ... 2.224e+06
    spatial_ref          int64 8B 0
Data variables:
    Optical_Depth_047    (y, x) int16 3MB ...
    Optical_Depth_055    (y, x) int16 3MB ...
    AOD_Uncertainty      (y, x) int16 3MB ...
    Column_WV            (y, x) int16 3MB ...
    AngstromExp_470-780  (y, x) int16 3MB ...
    AOD_QA               (y, x) uint16 3MB ...
    FineModeFraction     (y, x) float32 6MB ...
    Injection_Height     (y, x) float32 6MB ...
Attributes: (12/85)
    ADDITIONALLAYERS:                   3
    ALGORITHMPACKAGEACCEPTANCEDATE:     TBD
    ALGORITHMPACKAGEMATURITYCODE:       Preliminary
    ALGORITHMPACKAGENAME:               MOD_PR19
    ALGORITHMPACKAGEVERSION:            2.0
    ASSOCIATEDINSTRUMENTSHORTNAME.1:    MODIS
    ...                                 ...
    TileID:                             51009006
    TOTALADDITIONALOBSERVATIONS:        851908
    TOTALOBSERVATIONS:                  6837412
    VERSIONID:                          61
    VERTICALTILENUMBER:                 6
    WESTBOUNDINGCOORDINATE:             -103.923048441979

The source file contains metadata describing all available variables. You can choose to simplify this metadata by removing fields that are not relevant to your analysis and retaining only the most useful information.

In this example, we have chosen to keep all metadata fields for reference, but in practice, you may streamline it to focus on what matters for your specific workflow.

In [10]:
meta = ds[0].attrs
meta

{'ADDITIONALLAYERS': 3,
 'ALGORITHMPACKAGEACCEPTANCEDATE': 'TBD',
 'ALGORITHMPACKAGEMATURITYCODE': 'Preliminary',
 'ALGORITHMPACKAGENAME': 'MOD_PR19',
 'ALGORITHMPACKAGEVERSION': 2.0,
 'ASSOCIATEDINSTRUMENTSHORTNAME.1': 'MODIS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.2': 'MODIS',
 'ASSOCIATEDPLATFORMSHORTNAME.1': 'Terra',
 'ASSOCIATEDPLATFORMSHORTNAME.2': 'Aqua',
 'ASSOCIATEDSENSORSHORTNAME.1': 'MODIS',
 'ASSOCIATEDSENSORSHORTNAME.2': 'MODIS',
 'AUTOMATICQUALITYFLAG.1': 'Passed',
 'AUTOMATICQUALITYFLAGEXPLANATION.1': 'output file is created and good',
 'CHARACTERISTICBINANGULARSIZE': 30.0,
 'CHARACTERISTICBINSIZE': 926.625433055556,
 'DATACOLUMNS': 1200,
 'DATAROWS': 1200,
 'DAYNIGHTFLAG': 'Day',
 'DESCRREVISION': 6.1,
 'EASTBOUNDINGCOORDINATE': -85.1253536413533,
 'EQUATORCROSSINGDATE.1': '2025-03-01',
 'EQUATORCROSSINGLONGITUDE.1': -86.2825424063526,
 'EQUATORCROSSINGTIME.1': '15:15:12.078368',
 'EXCLUSIONGRINGFLAG.1': 'N',
 'GEOANYABNORMAL': 'False',
 'GEOESTMAXRMSERROR': 50.0,
 'GLOBALGRI

From metadata, `Orbit_time_stamp` are extracted to get the time associated to each orbit. The `T` at the end represents an orbit from `Terra` and `A` represents an orbit from `Aqua`. 

In [11]:
orbit_stamp = [item for item in ds[0].attrs['Orbit_time_stamp'].split(' ') if item != '']
orbit_stamp

['20250601510T', '20250601645T', '20250601945A', '20250602125A']

Next, let's list the available variables in the 1 km resolution grid.

In [12]:
variable = list(ds[0].data_vars.keys())
variable

['Optical_Depth_047',
 'Optical_Depth_055',
 'AOD_Uncertainty',
 'Column_WV',
 'AngstromExp_470-780',
 'AOD_QA',
 'FineModeFraction',
 'Injection_Height']

The map below visualizes the `Optical_Depth_047` data for all available orbits.

In [13]:
ds[0]['Optical_Depth_047'].sel(band=orbit_num).hvplot.image(x='x', y='y', aspect = 'equal', cmap='viridis')

BokehModel(combine_events=True, render_bundle={'docs_json': {'a284a2c7-2ff9-49ab-8106-b27926521a29': {'version…

### Convert Variables to COGs

At this stage, all variables available for each band and orbit can be extracted and saved as COGs.

If you intend to work with only a subset of variables, be sure to update the `variable` list accordingly to include only those you need, optimizing storage and processing efficiency.

In [14]:
for band in orbit_num:
    for var in variable:
        out = ds[0][var].sel(band=band)
        out.attrs['source'] = meta
        out.attrs['Orbit_time_stamp'] = orbit_stamp[band-1]

        filename = file.rsplit('\\')[-1].rsplit('.', 2)[0]
        out_filename = f"..\\data\\{file.rsplit('\\')[-1].rsplit('.', 2)[0]}.{orbit_stamp[band-1]}.{var}.orbit{band}.tif"
        # Write to COG
        out.rio.to_raster(out_filename, driver="COG")
        print(f'{out_filename} is stored successfully.')


..\data\MCD19A2.A2025060.h09v06.061.20250601510T.Optical_Depth_047.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.Optical_Depth_055.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.AOD_Uncertainty.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.Column_WV.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.AngstromExp_470-780.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.AOD_QA.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.FineModeFraction.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.Injection_Height.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601645T.Optical_Depth_047.orbit2.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601645T.Optical_Depth_055.orbit2.tif is stored successfully.
.

The same process can be applied to extract and store all variables from the 5 km spatial resolution grid.

In [15]:
for band in orbit_num:
    for var in list(ds[1].data_vars.keys()):
        out = ds[1][var].sel(band=band)
        out.attrs['source'] = ds[1].attrs
        out.attrs['Orbit_time_stamp'] = orbit_stamp[band-1]

        filename = file.rsplit('\\')[-1].rsplit('.', 2)[0]
        out_filename = f"..\\data\\{file.rsplit('\\')[-1].rsplit('.', 2)[0]}.{orbit_stamp[band-1]}.{var}.orbit{band}.tif"
        # Write to COG
        out.rio.to_raster(out_filename, driver="COG")
        print(f'{out_filename} is stored successfully.')


..\data\MCD19A2.A2025060.h09v06.061.20250601510T.cosVZA.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.RelAZ.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.Scattering_Angle.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.Glint_Angle.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601510T.cosSZA.orbit1.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601645T.cosVZA.orbit2.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601645T.RelAZ.orbit2.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601645T.Scattering_Angle.orbit2.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601645T.Glint_Angle.orbit2.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601645T.cosSZA.orbit2.tif is stored successfully.
..\data\MCD19A2.A2025060.h09v06.061.20250601945A.cosVZA.orbit3

You can now remove the source files to free up disk space. When working with files they often end up locked by `xarray`. **By restarting your kernel and running just this final cell, you can remove the source files.**

In [ ]:
# import os
# [os.remove(f'..\\data\\{f}') for f in os.listdir('..\\data') if f.endswith('.hdf')]

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://www.earthdata.nasa.gov/centers/lp-daac>  

¹Work performed under USGS contract 140G0121D0001 for NASA contract NNG14HH33I. 